# Cálculo de gradiente para uma rede linear

Este notebook implementa a equação 

\begin{equation*}
    \frac{\partial \mathcal{L}(x;\mathbb{W})}{\partial w_{ij}^l} = \nabla{\mathbf{a}_L}\mathcal{L}(x;\mathbb{W})W_L W_{L-1}\dots W_{l+1}\mathbf{\delta}_i^T a^{l-1}
\end{equation*}
deduzida nas notas de aula.

In [2]:
import torch
from torch import nn

@torch.no_grad
def calc_grad(model, x, i, j):
    '''Calcula o gradiente da primeira camada do modelo para o parâmetro na
     posição (i,j). Note que a primeira camada é a camada de entrada, portanto
    é necessário propagar os gradientes ao longo de toda a rede. É assumido 
    que o modelo é formado apenas por camadas lineares com bias=0.'''

    L = len(model) - 1
    # Assume que a loss foi calculada como loss = aL.sum(), e portanto o gradiente
    # da loss em relação à saída da rede é dado por um vetor de valores 1.
    grad_loss_aL = torch.ones(model[-1].weight.shape[0])
    v = grad_loss_aL
    for l in range(L, 0, -1):
        # Produto entre gradiente e matrizes Jacobianas
        v = v@model[l].weight
    # Seleção do resultado na posição i e multiplicação por x[j]
    grad_lij = v[i]*x[j]

    return grad_lij

def calc_grad_pytorch(model, x, i, j):
    '''Calcula o gradiente usando Pytorch.'''

    out = model(x)
    loss = out.sum()
    loss.backward()
    grad_lij = model[0].weight.grad[i,j]

    return grad_lij

# Modelo de exemplo
model = nn.Sequential(
    nn.Linear(10, 20, bias=False),
    nn.Linear(20, 30, bias=False),
    nn.Linear(30, 40, bias=False),
    nn.Linear(40, 50, bias=False),
    nn.Linear(50, 60, bias=False),
)
x = torch.rand(10)
grad_lij = calc_grad(model, x, i=4, j=6)
grad_lij_pt = calc_grad_pytorch(model, x, i=4, j=6)
# Compara os valores
print(grad_lij, grad_lij_pt)

tensor(-0.0414) tensor(-0.0414)
